In [1]:
import signdata
import importlib

import matplotlib.pyplot as plt
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from classifier import models, processor

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def normalize(array):
    _min, _max = array.min(), array.max()
    return (array - _min)/(_max - _min)

(data, idx) = signdata.load_train_data()
data = normalize(data.reshape(-1, 28, 28, 1))
labels = to_categorical(idx, num_classes=25)
print(data.shape)
print(labels.shape)

epochs = 50

(27455, 28, 28, 1)
(27455, 25)


In [7]:
importlib.reload(models)

model = models.SunGod()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_14 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 22, 22, 128)       73856     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 22, 22, 128)       16512     
__________

In [3]:
importlib.reload(models)
importlib.reload(processor)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(data)

# fits the model on batches with real-time data augmentation:
model.model.fit_generator(datagen.flow(data, labels, batch_size=32),
                    steps_per_epoch=len(data) / 32, epochs=15)

processor.save_keras(model.name, model.model, 1)

# here's a more "manual" example
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(data, labels, batch_size=32):
        model.model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / 32:
            break


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 26, 26, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)       16512     
__________

KeyboardInterrupt: 

In [ ]:
(testData, idx) = signdata.load_test_data()
testData = testData.reshape(-1, 28, 28, 1)
testLabels = to_categorical(idx, num_classes=25)
print(testData.shape)
print(testLabels.shape)

In [ ]:
model.model.evaluate(testData, testLabels)

In [ ]:
y = model.model.predict(testData)
accuracy = np.sum([actual == expected for actual, expected in zip(y, testLabels)]) / len(y)